# ICL data science challenge

## EDA

In [4]:
import pandas as pd
import numpy as np
import re

In [6]:
train_path = "../input/raw/train.csv"
preprocessed_path = "../input/derived/train-preprocessed.csv"
cols = None
data_lines = []

with open(train_path, "rt") as fin:
    with open(preprocessed_path, "wt") as fout:
        for line in fin:
            fout.write(line.replace('\\"', "'"))


# with open(train_path, "r") as f:
#     for i, line in enumerate(f):
#         if i == 0:
#             cols = [re.sub('\s+', '', x.strip('"')).strip('"') for x in line.split(" ")]
#         else:
#             data_lines.append(line.split(' '))
#             #print(len(data_lines[-1]))

# print(len(cols))

In [7]:
df = pd.read_csv(preprocessed_path, sep=r'\s+', quotechar='"')
df

,Index,Artist,Track,Album,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,AnC,AnD,BnC,BnD,CnD,AnBnC,AnBnD,AnCnD,BnCnD,AnBnCnD
0,1,tajima hal,Grape Choice,Grape Choice,0.704,0.5870,-9.815,0.3120,0.341,0.004510,...,0,0,0,0,0,0,0,0,0,0
1,2,CHROMANCE,Wrap Me In Plastic - Marcus Layton Radio Edit,Wrap Me In Plastic (Marcus Layton Radio Edit),0.917,0.6120,-2.183,0.0854,0.238,0.000023,...,0,0,0,0,0,0,0,0,0,0
2,3,Stevie Wonder,Sir Duke,Songs In The Key Of Life,0.588,0.5680,-9.766,0.0672,0.160,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,4,MishCatt,Midnight Sun - Acoustic Version,Midnight Sun (Acoustic Version),0.590,0.0991,-15.925,0.0490,0.915,0.000005,...,0,0,0,0,0,0,0,0,0,0
4,5,Gloria Estefan,Sólo Por Tu Amor,Alma Caribeña,0.561,0.4770,-8.123,0.0355,0.295,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7588,7589,Some Sprouts,Someone You Love,Divinely Uninspired To A Hellish Extent,0.501,0.4050,-5.679,0.0319,0.751,0.000000,...,0,0,0,0,0,0,0,0,0,0
7589,7590,Sure Sure,Funky Galileo,Funky Galileo,0.802,0.3250,-9.327,0.0466,0.164,0.162000,...,0,0,0,0,0,0,0,0,0,0
7590,7591,The Flying Stars Of Brooklyn NY,Live On,My God Has a Telephone,0.513,0.6260,-8.843,0.0582,0.579,0.000633,...,0,0,0,0,0,0,0,0,0,0
7591,7592,Gasper Lawal,Kita Kita,Nigeria 70 - Funky Lagos,0.614,0.9070,-6.565,0.0472,0.062,0.772000,...,0,0,0,0,0,0,0,0,0,0


## Sanity checking cells

In [51]:
temp = ['AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD',
        'AnBnC', 'AnBnD', 'AnCnD', 'BnCnD', 'AnBnCnD']
for i in range(len(df['A'])):
    for c in temp:
        row = df.iloc[i]
        if row[c] == 1:
            if 'A' in c:
                if row['A'] == 0:
                    print(row['Index'], 'A')
            if 'B' in c:
                if row['B'] == 0:
                    print(row['Index'], 'B')
            if 'C' in c:
                if row['C'] == 0:
                    print(row['Index'], 'C')
            if 'D' in c:
                if row['D'] == 0:
                    print(row['Index'], 'D')

234 A
330 B
752 A
2016 B
2261 A
2419 A
3531 B
3863 B
3873 B
3873 B
4375 A
4997 B
5230 B
5466 B
6337 A
6362 A
6627 B
7372 B
7432 A


In [55]:
df.columns

Index(['Index', 'Artist', 'Track', 'Album', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'length', 'time_signature', 'A', 'B', 'C', 'D',
       'AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD', 'AnBnC', 'AnBnD', 'AnCnD',
       'BnCnD', 'AnBnCnD'],
      dtype='object')

## Baseline model logistic regression

In [8]:
df.columns
feature_cols = ['danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'length']
target_cols = ['A', 'B', 'C', 'D']

X = df[feature_cols].to_numpy()
y = df[target_cols].to_numpy()

In [61]:
X.shape, y.shape

((7593, 10), (7593, 4))

In [69]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

logRegs = [LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression()]
accs = []

for i in range(4):
    logRegs[i].fit(X_train, y_train[:, i])
    preds = logRegs[i].predict(X_test)
    acc = accuracy_score(y_test[:, i], preds)
    accs.append(acc)
    print(f"Validation accuracy for class {target_cols[i]}: {acc:.4f}")
print(f"Average validation accuracy: {np.mean(accs)}")

Validation accuracy for class A: 0.6504
Validation accuracy for class B: 0.6261
Validation accuracy for class C: 0.9835
Validation accuracy for class D: 0.8308
Average validation accuracy: 0.7727123107307439


In [70]:
from src.preprocessing.data_formatting import get_preprocessed_data
# produce predictions on test data
df_test = get_preprocessed_data("../input/", skip_preprocess=False, get_test=True)

X_test = df_test[feature_cols].to_numpy()
for i in range(4):
    preds = logRegs[i].predict(X_test)
    df_test[target_cols[i]] = preds

In [60]:
df_test['A'].mean()

0.04636459430979979

In [73]:
def fill_remaining_cols(df):
    extended_cols = ['AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD', 'AnBnC', 'AnBnD', 'AnCnD', 'BnCnD', 'AnBnCnD']
    for i in range(len(df['A'])):
        for c in extended_cols:
            fill_val = True
            for l in ['A', 'B', 'C', 'D']:
                if l in c:
                    fill_val = fill_val and (df.at[i, l] == 1)
            # modify the cell
            df.at[i, c] = int(fill_val)

def submit_dataframe(df, submit_path):
    df = df.reset_index()
    assert len(df.columns) == 29
    df_test.to_csv(submit_path, index=False, quotechar='"', sep=' ')

submit_dataframe(df_test, "../output/2sigma.csv")

In [63]:
len(df_test.columns)

28